<a href="https://colab.research.google.com/github/Akshay-Kumar-Arya/Identify_the_sentiments/blob/master/nnlm_logistic_reg_model_evaluation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logistic regression using nnlm vectors

In [1]:
import sklearn
from sklearn.model_selection import train_test_split
import pickle
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

pd.set_option('display.max_colwidth', 200)

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
# Data path
training_data_path = "/content/gdrive/My Drive/Identify_the_sentiments/train.csv"
test_data_path =  "/content/gdrive/My Drive/Identify_the_sentiments/test.csv"

save_path = "/content/gdrive/My Drive/Identify_the_sentiments/"

In [4]:
# reading data from csv
train_data = pd.read_csv(training_data_path)
test_data = pd.read_csv(test_data_path)

print (train_data.shape, test_data.shape)

(7920, 3) (1953, 2)


In [5]:
# import bert embeddings for tweets
train_file = open(save_path + "nnlm_train.pickle", "rb")
pool_embs_train = pickle.load(train_file)

test_file = open(save_path + "nnlm_test.pickle", "rb")
pool_embs_test = pickle.load(test_file)

In [7]:
# splitting training and validation data
xtrain, xvalid, ytrain, yvalid = train_test_split(pool_embs_train, 
                                                  train_data['label'],  
                                                  random_state=42, 
                                                  test_size=0.2)
print (xtrain.shape, xvalid.shape, ytrain.shape, yvalid.shape)

(6336, 128) (1584, 128) (6336,) (1584,)


In [10]:
# logistic regression model
lreg = LogisticRegression(max_iter = 10000, warm_start= True, verbose=True)
lreg.fit(xtrain, ytrain)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=True,
                   warm_start=True)

In [11]:
# prediction on validation dataset
preds_valid = lreg.predict(xvalid)
print (f1_score(yvalid, preds_valid))

0.7686746987951807


In [12]:
# prediction on test dataset
preds_test = lreg.predict(pool_embs_test)


# prepare submission dataframe
sub = pd.DataFrame({'id':test_data['id'], 'label':preds_test})

# write predictions to a CSV file
sub.to_csv(save_path + "sub_lreg_nnlm.csv", index=False)
# Your score for this submission is : 0.85817439707144.